In [1]:
from Scrapper import *

In [20]:
search= "shopplanet13"

In [19]:
username = 'notarobot8591'
password = 'Lucille1515'


#Options for our driver
options =webdriver.ChromeOptions()
#options.add_argument('headless')
options.add_argument('--disable-blink-feature=AutomationControlled')
options.add_argument('--flags#enable-new-usb-backend')
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)

#Creating our driver object
global driver
driver = webdriver.Chrome(options = options, service=Service(ChromeDriverManager().install()))
driver.implicitly_wait(5)
stealth(driver,
languages=["en-US", "en"],
vendor="Google Inc.",
platform="Win32",
webgl_vendor="Intel Inc.",
renderer="Intel Iris OpenGL Engine",
fix_hairline=True,
)
#Opens browser and logins in
driver.get('http://www.instagram.com')

#Login information entered into text boxes
driver.find_element(By.NAME,'username').send_keys(username)
driver.find_element(By.NAME, 'password').send_keys(password)
tm.sleep(1)
driver.find_element(By.XPATH,'//div[contains(text(),"Log in")]').click()

#Not now notifications
driver.find_element(By.XPATH, '//*[contains(text(),"Not Now")]').click()
driver.find_element(By.XPATH, '//*[contains(text(),"Not Now")]').click()
        
driver.find_element(By.XPATH,'//div[contains(text(),"Search")]').click()
ActionChains(driver).send_keys(search).perform()
tm.sleep(1)
ActionChains(driver).send_keys(Keys.ENTER).perform()






====== WebDriver manager ======
Current google-chrome version is 107.0.5304
Get LATEST chromedriver version for 107.0.5304 google-chrome
Driver [C:\Users\bboul\.wdm\drivers\chromedriver\win32\107.0.5304.62\chromedriver.exe] found in cache


In [18]:
data = {'Time':[],'Post':[],'Image':[],'Caption':[],'Likes':[],'Hashtags':[]}

In [21]:
amount= 500

for i in range(amount):
    img_box = driver.find_elements(By.XPATH, value = '//div[@class="_aagv"]/img')
    img=[img.get_attribute('alt') for img in img_box]
    try:
        driver.find_element(By.XPATH, value = '//div[@class="_9zm2"]')
        if any("Photo by" in s for s in img):
            data['Image'].append(img[-2:])
        else:
            data['Image'].append('Video')
            
    except NoSuchElementException:
            
        if any("Photo by" in s for s in img):
            data['Image'].append(img[-1])
        else:
            data['Image'].append('Video')
            
    #Post URL
    post = driver.current_url
    data['Post'].append(post)
            
    #Scraping likes
    try:
        likes = driver.find_element(By.XPATH, value = '//div[@class="_aacl _aaco _aacw _aacx _aada _aade"]/span')
        likes = likes.text
    except NoSuchElementException:
        likes = driver.find_elements(By.XPATH, value = '//div[@class="_aacl _aaco _aacw _aacx _aad6 _aade"]')[-1].click()
    except NoSuchElementException:
        likes = driver.find_element(By.XPATH, value = '//div[@class="_aauu"]/span')
        likes = likes.text
    except NoSuchElementException:
        likes = 'Not Displayed'
    finally: 
        data['Likes'].append(likes) 
        
    #Scrape Hashtags
    hashtags = driver.find_elements(By.XPATH, value = '//a[@class="x1i10hfl xjbqb8w x6umtig x1b1mbwd xaqea5y xav7gou x9f619 x1ypdohk xt0psk2 xe8uvvx xdj266r x11i5rnm xat24cr x1mh8g0r xexx8yu x4uap5 x18d9i69 xkhd6sd x16tdsg8 x1hl2dhg xggy1nq x1a2a7pz  _aa9_ _a6hd"]')
    hashtags = [hashtag.text for hashtag in hashtags]
    data['Hashtags'].append(hashtags)
            
    #Scrape Timestamp
    date_location = driver.find_element(By.TAG_NAME, value = 'time')
    time_attribute = date_location.get_attribute('datetime')
    data['Time'].append(time_attribute)
            
    #Scrape post caption
    try:
        text_box = driver.find_element(By.XPATH, value = '//div[@class="_a9zs"]/span')
        caption = text_box.text
        data['Caption'].append(caption)
        #Scraping image data
    except NoSuchElementException:
        pass

    #Clicking Next post and sleeping for (1-3) seconds
    tm.sleep(randint(1,3)) 
    ActionChains(driver).key_down(Keys.ARROW_RIGHT).key_up(Keys.ARROW_RIGHT).perform()

In [ ]:
# Libaries needed
import json
import requests
import numpy as np
import pandas as pd 
import regex as re
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import string
from pandas.api.types import CategoricalDtype
import time

#Temporary Dictionary
url = 'https://stlouis.myfeelstate.com/graphql?operationName=FilteredProducts&variables={"includeEnterpriseSpecials":false,"includeCannabinoids":true,"productsFilter":{"dispensaryId":"600f5a29d692585e7ffd25e4","pricingType":"med","strainTypes":[],"subcategories":[],"Status":"Active","types":[],"useCache":false,"sortDirection":1,"sortBy":null,"isDefaultSort":true,"bypassOnlineThresholds":false,"isKioskMenu":false,"removeProductsBelowOptionThresholds":true},"page":0,"perPage":500}&extensions={"persistedQuery":{"version":1,"sha256Hash":"aed681974c9093e0cc3099ef18a8d37e17e4916a9f7743dd372e0be6b13c8f10"}}'
menu_dictionary = {'Type':[],'Brand':[],'Name':[],'Weight':[], 'THC_content':[], 'CBD_content':[],'Price':[]}
# While loop to retrieve data

#function for making get request to Dutchie API 
def retrieve_data(url):
    r= requests.get(url)
    data = r.json()
    global product_data
    product_data= data['data']['filteredProducts']['products']

# Flower,Pre-Rolls,Concentrate,Vaporizers these are to be displayed showing percentage of thc and cbd
# Tincture,Topicals,CBD,Edible these are to be displayed showing miligrams(mg) of thc and cbd
# Functions to retrieve cleand and upload data to dictionary 
def retrieve_name(item):
    # parses json for the name of product and removes the weight text from the name 
    name = re.sub(weight, '', product)
    menu_dictionary['Name'].append(name.strip())
    menu_dictionary['Weight'].append(weight)


def retrieve_weight_gram(item):
    # assigns weight unit variable from json and weight amount, divides the weight to keep values uniform
    unit_dictionary= {'Milligrams':'g'}
    weight_unit= item['measurements']['netWeight']['unit'].title()
    weight =  str(item['measurements']['netWeight']['values'][0]/1000)+unit_dictionary.get(weight_unit)
    menu_dictionary['Weight'].append(weight)
   
def retrieve_weight_mg(item):  
    # assigns weight unit variable from json and weight amount, concats suffix to measurement value
    unit_dictionary_2= {'Percentage':'%','Milligrams':'mg','Milliliters':'ml'}
    weight_unit= item['measurements']['netWeight']['unit'].title()
    weight=''.join(str(item['measurements']['netWeight']['values']).strip('[]')+unit_dictionary_2.get(weight_unit))
    menu_dictionary['Weight'].append(weight)

def CBD_percentage(item):
    # if statment to verify product has CDB content and larger than 0
    # concats "mg" to value appends to dictionary
    if item['CBDContent'] == None:
        menu_dictionary['CBD_content'].append(' ')
    elif item['CBDContent']['range'] != None and item['CBDContent']['range'][-1] > 0:
        CBD_content =  str(item['CBDContent']['range'][-1]).strip('[]')+'%'
        menu_dictionary['CBD_content'].append(CBD_content)
    else:
        menu_dictionary['CBD_content'].append(' ')

def CBD_content(item):
     # if statment to verify product has CDB content and larger than 0
     #concats "mg" to value appends to dictionary
    if (item['CBDContent']['range'] != None) and (item['CBDContent']['range'][-1] > 0):
        CBD_content =  str(item['CBDContent']['range'][-1]).strip('[]')+'mg'
        menu_dictionary['CBD_content'].append(CBD_content)
    else:
        menu_dictionary['CBD_content'].append(' ')
        
def THC_percentage(item):
    # if statment to verify product has THC content and larger than 0
    # concats "mg" to value appends to dictionary
    if item['THCContent']['range'] != None and (item['THCContent']['range'][-1] > 0):
        THC_content = str(item['THCContent']['range'][-1]).strip('[]')+'%'
        menu_dictionary['THC_content'].append(THC_content)
    else:
         menu_dictionary['THC_content'].append(' ')


def THC_content(item):
    # if statment to verify product has THC content and larger than 0
    # concats "mg" to value appends to dictionary
    if item['THCContent']['range'] != None and (item['THCContent']['range'][-1] > 0):
        THC_content = str(item['THCContent']['range'][-1]).strip('[]')+'mg'
        menu_dictionary['THC_content'].append(THC_content)
    else:
         menu_dictionary['THC_content'].append(' ')


def retrieve_price(item):
    global price
    #if statement to verify if price is float or integar
    #Price needs to be displayed with decimal only when required example $21.50 example $21
    if isinstance(max(item['Prices']), float):
        price = '{:.2f}'.format(max(item['Prices']))
        menu_dictionary['Price'].append('$'+ price)
    else:
        price = max(item['Prices'])
        menu_dictionary['Price'].append('$'+str(price))


# Sets range of table to be used with google sheets api
def get_range(df):
    range_end= [(i,word) for i,word in zip(string.ascii_uppercase, df.columns.values)][-1][0]
    global table_range
    table_range = f'A:{range_end}'

# Function to sort data according to menu layout 
def custom_sort(df):
    house_brands=['Heya','Robust','Lush Labs','Packs Cannabis']
    df.fillna('', inplace=True)
    # How the Categories are to be displayed 
    sort_types = CategoricalDtype(['Vaporizers','Edible','Concentrate','Topicals','Tincture','CBD','Accessories','Apparel'], ordered =True)
    
    # Seperates house brand flowers from non house brands(n_h_flower) sorts them and concats them back together
    house_flower = df.loc[(df['Type'] == 'Flower')&(df['Brand'].isin(house_brands))].sort_values(by=['Brand','Name'], ascending=[False, True])
    n_h_flower = df.loc[(df['Type'] == 'Flower')&(~df['Brand'].isin(house_brands))].sort_values(by=['Brand','Name'])
    flower = pd.concat([house_flower, n_h_flower])

    # Seperates house brand prerolls from non house prerolls(n_h_prerolls) sorts them and concats them back together
    house_prerolls = df.loc[(df['Type'] == 'Pre-Rolls')&(df['Brand'].isin(house_brands))].sort_values(by=['Brand','Name'])
    n_h_prerolls = df.loc[(df['Type'] == 'Pre-Rolls')& (~df['Brand'].isin(house_brands))].sort_values(by=['Brand','Name'])
    prerolls = pd.concat([house_prerolls, n_h_prerolls])

     # Seperates house brand prerolls from non house vaporizers sorts them and concats them back together
    house_vapes = df.loc[(df['Type'] == 'Vaporizers')&(df['Brand'].isin(house_brands))].sort_values(by=['Brand','Name'])
    n_h_vapes = df.loc[(df['Type'] == 'Vaporizers')& (~df['Brand'].isin(house_brands))].sort_values(by=['Brand','Name'])
    vapes = pd.concat([house_vapes, n_h_vapes])

    # Items to be dropped by index
    drop_list = list(flower.index) + list(prerolls.index) + list(vapes.index)
    non_house_brands = df.drop(labels=drop_list)
    non_house_brands['Type'] = non_house_brands['Type'].astype(sort_types)
    
    #sorts table according to Type Then Brand then Name
    non_house_brands= non_house_brands.sort_values(by=['Type','Brand','Name'])

    #concates new sorted categories house brands and non house brands
    clean_data= pd.concat([flower,prerolls,vapes,non_house_brands]).applymap(str)
    clean_data= clean_data.drop_duplicates()

    global menu
    #creates menu variable and ransform data to correct format for googlesheets api 
    menu = clean_data.values.tolist()
    print(menu)

# Main loop that iterates over JSON reponse and filters data
retrieve_data(url)
for item in product_data:
    # Types are used to separate how they will be displayed 
    gram_type= ['Flower','Pre-Rolls','Concentrate','Vaporizers']
    mg_type= ['Tincture','Topicals','CBD','Edible']
    
    # Removing brand from name category 
    brand = str(item['brandName'])
    product= str(item['Name']).strip()
    product= re.sub(brand, '', product)
    product= re.sub('[-|]', '', product)

    # Adding type, brand to dictionary 
    menu_dictionary['Type'].append(item['type'])
    menu_dictionary['Brand'].append(brand)
    
    # Functions to clean data then append to dictionary
    retrieve_price(item)

    #Searching name for weight to remove and use as weight
    weight = re.search('\d.\d+g$|\d+g$|\d*mg$|^ \d+.*ck$|\d+ml$', product)
    
    #If statements are based on menu perferences on how menu item are to be displayed (GRAMS vs MILIGRAMS)
    #If producti is to be displayed using grams 
    if (weight != None) and (item['type'] in gram_type):
        weight = weight.group() 
        retrieve_name(item)
        CBD_percentage(item)
        THC_percentage(item)
    # If product is to be displayed using miligrams     
    elif (weight != None) and (item['type'] in mg_type):
        weight = weight.group() 
        retrieve_name(item)
        CBD_content(item)
        THC_content(item)
    # If product CBD and THC content to be displayed in garams 
    elif (item['type'] in gram_type) and (item['measurements']['netWeight']['values'] != []):
        menu_dictionary['Name'].append(product)
        retrieve_weight_gram(item)
        CBD_percentage(item)
        THC_percentage(item)
    # If product CBD and THC content to be displayed in milligarams 
    elif (item['type'] in mg_type) and (item['measurements']['netWeight']['values'] != []):
        menu_dictionary['Name'].append(product)
        retrieve_weight_mg(item)
        CBD_content(item)
        THC_content(item) 
    # Other products such as accessories and apparel do not receive THC or CBD content   
    else:
        menu_dictionary['THC_content'].append(' ')
        menu_dictionary['CBD_content'].append(' ')
        menu_dictionary['Weight'].append(' ')
        menu_dictionary['Name'].append(product.strip())
# Converting dictionary to pandas dataframe    
df =pd.DataFrame.from_dict(data=menu_dictionary)
get_range(df)
custom_sort(df)

# Using json API Key to make connection to google sheets using gspread python library
# json_key will be saved to client computer ACTION REQURIED must insert json key into '' 
client = gspread.service_account(r'C:\Users\bboul\Desktop\feel_state_key.json')
# Opening workbook
workbook= client.open('Feel_State')
# Opening First Worksheet
worksheet = workbook.get_worksheet(0)
# Clearing the Sheet 
worksheet.clear()
# Updating new Data
worksheet.update(table_range, menu)


In [24]:
dataframe = pd.DataFrame.from_dict(data)

In [23]:
print(search)

shopplanet13


In [25]:
dataframe.to_csv(f'C:/Users/bboul/OneDrive/Documents/feel_state_data/{search}_{amount}.csv', index=False)
print('Post Scrapped: '+str(len(dataframe.index)))

Post Scrapped: 500


In [22]:

for i in data:
    print(i)
    print(len(data[i]))

Time
500
Post
500
Image
500
Caption
500
Likes
500
Hashtags
500


In [24]:
data['Hashtags'].pop()

[]

In [11]:
data['Caption'].insert(4, "No Caption")

In [13]:
for i in data['Caption']:
    print(i)

Have you caught the Good Tide yet? 🌊
A tropical getaway to escape the gloomy weather in 3 delicious flavors 🍍
.
.
.
-not for sale-
-keep out of reach of children-
BOO! 7 DAYS OF TREATS (no tricks)😱
.
.
.
-not for sale-
-keep out of reach of children-
Let us read your fortune… you’re going to be high for Halloween🔮
.
.
.
-not for sale-
-keep out of reach of children-
BOO! 7 DAYS OF TREATS (no tricks)😱
.
.
.
-not for sale-
-keep out of reach of children-
No Caption
🦴 fresh drop 🦴
.
.
-keep out of reach of children-
.
.
.
@doghouse_fl
hot!🔥 (noxious may be out next week, if you can handle it)
.
.
.
-keep out of reach of children-
Big shoutout to one of our favorites in the industry— Dave, from @deepcreekgardens !

Dave was repping OF when he was playing at USTA National Tennis Championships in AL two weeks ago.

Tag us in your OF Gear action shots 🎾🏂🚴🏌️🏋️‍♂️⛹️‍♂️ 🏓
(hint: there may be a summertime giveaway in the works with a basket of that good good for the winner)
.
.
.
-keep out of rea

In [8]:
dataframe.head()

,Time,Post,Image,Caption,Likes,Hashtags
0,2022-11-13T16:08:19.000Z,https://www.instagram.com/p/Ck6EG-Mu2ac/,Photo by @feelstateflorissant in Feel State Fl...,Heavenly Sativa from @robust_cannabis ;),25,[]
1,2022-11-11T20:15:26.000Z,https://www.instagram.com/p/Ck1Wo-Ju1VW/,Photo shared by @feelstateflorissant on Novemb...,Please join us in thanking three very importan...,50,[]
2,2022-11-10T02:13:53.000Z,https://www.instagram.com/p/CkwX64oOz9Y/,Photo by @feelstateflorissant in Feel State Fl...,This Friday we welcome @kosmikbrands to table ...,27,[]
3,2022-11-08T22:47:27.000Z,https://www.instagram.com/p/Ckt5nUxOBBq/,Photo by @feelstateflorissant in Feel State Fl...,A budtender favorite this week\n\n#reels#trend...,32,"[#reels, #trending, #fyp, #artisansofdank, #am..."
4,2022-11-08T16:56:01.000Z,https://www.instagram.com/p/CktNbhPLyJV/,Photo by @feelstateflorissant in Feel State Fl...,@airo.brands is changing things up a bit! In t...,16,[#vapecartridges]


In [ ]:
Insta_Bot()